In [1]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 75%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [1]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# tensorflow Module
import tensorflow as tf
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import losses

# python basic Module
import os
import sys
import types
from datetime import datetime
from shutil import copy
from pickle import dump, load

# math, image, plot Module
import numpy as np
import cv2
import matplotlib.pyplot as plt  # TBD

# email Module
import smtplib
from email.mime.text import MIMEText
from email.header import Header

from data_loader.medical_segmentation_data_loader_v1 import DataLoader

from gan_module.model import build_generator, build_discriminator
from gan_module.draw_images import ImageDrawer
from gan_module.custom_loss import f1_loss_for_training, f1_score, dice_loss_for_training

# set GPU memory growth allocation
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

"""
if you don't have nvidia-gpu, try plaidml! but it will works tensorflow 1.x.x
# pip install -U plaidml-keras
# plaidml-setup
"""
# use_plaidml = False
# if use_plaidml :
#     import plaidml.keras
#     plaidml.keras.install_backend()
#     os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


# class CaptchaDiscriminator
# captcha_generator = CaptchaGenerator()
# CaptchaGeneratorTrainer(captcha_generator, discriminator)
# class CaptchaGeneratorTrainer():
# class LearningRate():
# class Smtp()
# class Shape()
# class InputShape(Shape):
# class OutputShape(Shape):
# class Model():
#     def save():
#     def load():
# class Generator(Model) :
#     @override
#     def save():
#         super()
# class HyperParameter()


class Pix2PixSegmentation:
    def __init__(
        self,
        generator_power=32,
        discriminator_power=32,
        generator_learning_rate=1e-4,
        discriminator_learning_rate=1e-4,
        learning_rate_decay_rate_epoch=0.1,
        learning_rate_decay_rate_dynamic=0.1,
        find_init_epoch=150,
        find_error=False,
        temp_weights_path=".",
        draw_images=True,
        on_memory=True,
        test=False
    ):
        # smtp info
        self.smtp_host = "smtp.gmail.com"
        self.smtp_port = 465
        self.smtp_id = "rpa.manager0001@gmail.com"
        self.smtp_password = "!rpa.admin!23"
        self.smtp_to_addr = "tobeor3009@gmail.com"

        # Input shape
        self.img_rows = 512
        self.img_cols = 512
        self.input_channels = 3
        self.output_channels = 1
        self.input_img_shape = (
            self.img_rows, self.img_cols, self.input_channels)
        self.output_img_shape = (
            self.img_rows, self.img_cols, self.output_channels)
        # set parameter
        self.start_epoch = None
        self.history = [[], [], []]
        self.f1_loss_ratio = 25
        self.huber_loss_ratio = 100
        self.learning_rate_decay_rate_epoch = learning_rate_decay_rate_epoch
        self.learning_rate_decay_rate_dynamic = learning_rate_decay_rate_dynamic
        self.find_init_epoch = find_init_epoch
        self.find_error = find_error
        self.find_error_epoch = 30
        self.error_list = []
        self.temp_weights_path = temp_weights_path

        # Configure data loader
        self.dataset_name = "tumor"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            img_res=(self.img_rows, self.img_cols),
            on_memory=on_memory, test=test
        )
        self.train_loaded_data, self.valid_loaded_data = self.data_loader.load_all()
        if test:
            self.train_loaded_data_len = 20
            self.valid_loaded_data_len = 20
        else:
            self.train_loaded_data_len = self.data_loader.train_data_length
            self.valid_loaded_data_len = self.data_loader.valid_data_length

        self.train_loaded_data_index = np.arange(self.train_loaded_data_len)
        self.valid_loaded_data_index = np.arange(self.valid_loaded_data_len)

        # Configure Image Drawer
        self.draw_images = draw_images
        self.image_drawer = ImageDrawer(
            dataset_name=self.dataset_name, data_loader=self.data_loader
        )
        # training parameters
        self.learning_schedule = [
            50,
            100,
            150,
            200,
            250,
            300,
            350,
            400,
            450,
            500,
            550
        ]
        self.discriminator_acc_previous = 0.5
        self.discriminator_acc_high_indexes = np.array([0.5 for _ in range(self.train_loaded_data_len)])
        self.discriminator_acc_high_indexes_previous = self.discriminator_acc_high_indexes.copy()
        self.generator_loss_min = 100
        self.generator_loss_previous = 100
        self.generator_loss_max_previous = 1000
        self.generator_loss_max_min = 1000
        self.generator_loss_min_min = 1000
        self.weight_save_stack = False
        self.weight_stagnant_stack = 0
        self.look_on_for_generator_min_loss = False
        self.learning_rate_decay_dynamic = 0
        self.training_end_stack = 0
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 2)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.generator_power = generator_power
        self.discriminator_power = discriminator_power
        self.generator_learning_rate = generator_learning_rate
        self.discriminator_learning_rate = discriminator_learning_rate
        generator_optimizer = Nadam(self.generator_learning_rate)
        discriminator_optimizer = Nadam(self.discriminator_learning_rate)

        # layer Component
        self.kernel_initializer = RandomNormal(mean=0.0, stddev=0.02)

        # Build and compile the discriminator
        self.discriminator = build_discriminator(
            input_img_shape=self.input_img_shape,
            output_img_shape=self.output_img_shape,
            discriminator_power=self.discriminator_power,
            kernel_initializer=self.kernel_initializer,
        )
        # self.discriminator = self.build_discriminator()
        # 'mse' or tf.keras.losses.Huber() tf.keras.losses.LogCosh()
        self.discriminator.compile(
            loss=tf.keras.losses.LogCosh(),
            optimizer=discriminator_optimizer,
            metrics=["accuracy"],
        )

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Build the generator
        self.generator = build_generator(
            input_img_shape=self.input_img_shape,
            output_channels=self.output_channels,
            generator_power=self.generator_power,
            kernel_initializer=self.kernel_initializer,
        )

        # Input images and their conditioning images
        original_img = Input(shape=self.input_img_shape)
        masked_img = Input(shape=self.output_img_shape)
        # generate image from original_img for target masked_img
        model_masked_img = self.generator(original_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        # Discriminators determines validity of translated images / condition pairs
        model_validity = self.discriminator([original_img,model_masked_img])
        # give score by
        # 1. how generator trick discriminator
        # 2. how generator's image same as real photo in pixel
        # 3. if you want change loss, see doc https://keras.io/api/losses/
        # 4. 'mse', 'mae', tf.keras.losses.LogCosh(),  tf.keras.losses.Huber()
        self.combined = Model(
            inputs=[original_img, masked_img],
            outputs=[model_validity, model_masked_img, model_masked_img],
        )
        self.combined.compile(
            loss=[
                tf.keras.losses.LogCosh(),
                tf.keras.losses.Huber(),
                dice_loss_for_training
            ],
            loss_weights=[0.5, self.huber_loss_ratio, self.f1_loss_ratio],
            optimizer=generator_optimizer
        )

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.now()

        # Adversarial loss ground truths
        self.training_end_stack = 0
        self.batch_size = batch_size
        valid_patch = np.ones((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        fake_patch = np.zeros((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        if self.start_epoch is None:
            self.start_epoch = 0
        for epoch in range(self.start_epoch, epochs):
            batch_i = 0
            discriminator_losses = []
            discriminator_acces = []
            train_generator_loss = []
            generator_loss_max_in_epoch = 0
            generator_loss_min_in_epoch = 1000
            generator_current_learning_rate = self.learning_rate_scheduler(
                self.generator_learning_rate
                * (2 ** (self.learning_rate_decay_dynamic)),
                epoch,
            )
            discriminator_current_learning_rate = self.learning_rate_scheduler(
                self.discriminator_learning_rate
                * (2 ** (self.learning_rate_decay_dynamic)),
                epoch,
            )
            keras_backend.set_value(
                self.combined.optimizer.learning_rate, generator_current_learning_rate
            )
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            # shffle data 5 epoch term
            if epoch % 5 == 0 and not isinstance(self.train_loaded_data, types.GeneratorType):
                np.random.shuffle(self.train_loaded_data_index)
                self.train_loaded_data = [
                    self.train_loaded_data[i][self.train_loaded_data_index] for i in range(2)
                ]
                self.discriminator_acc_high_indexes_previous = self.discriminator_acc_high_indexes_previous[self.train_loaded_data_index]
            
            while batch_i + self.batch_size <= self.train_loaded_data_len:
                original_img = self.train_loaded_data[0][batch_i: batch_i +
                                                         self.batch_size]
                masked_img = self.train_loaded_data[1][batch_i: batch_i +
                                                       self.batch_size]
                model_masked_img = self.generator.predict_on_batch(
                    original_img)

                # forTest
                self.masked_img = masked_img
                self.original_img = original_img
                self.model_masked_img = model_masked_img
                self.valid_path = valid_patch
                self.fake_patch = fake_patch
                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Condition on B and generate a translated version

                # Train the discriminators (target image = masked_img / generated_img = model_masked_img)
                self.discriminator.train_on_batch([original_img, masked_img], valid_patch)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                generator_loss = self.combined.train_on_batch(
                    [original_img, masked_img],
                    [valid_patch, masked_img, masked_img],
                )
                # train images discriminator failed to catch
                if np.mean(self.discriminator_acc_high_indexes_previous[batch_i : batch_i + self.batch_size] <= 0.5:
                    discriminator_loss = self.discriminator.train_on_batch([original_img, model_masked_img], fake_patch)
                else:
                    discriminator_loss = self.discriminator.test_on_batch([original_img, model_masked_img], fake_patch)
                self.discriminator_acc_high_indexes[batch_i : batch_i + self.batch_size] = discriminator_loss[1]
                elapsed_time = datetime.now() - start_time
                if batch_i % sample_interval == 0:
                    # Plot the progress
                    print(
                        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s"
                        % (
                            epoch,
                            epochs,
                            batch_i,
                            self.train_loaded_data_len,
                            discriminator_loss[0],
                            100 * discriminator_loss[1],
                            generator_loss[0],
                            elapsed_time,
                        )
                    )

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0 and self.draw_images:
                    self.image_drawer.sample_images(
                        self.generator, epoch, batch_i)
                
                discriminator_losses.append(discriminator_loss[0])
                discriminator_acces.append(discriminator_loss[1])
                train_generator_loss.append(generator_loss[0])
                # loss 가 가장 높은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss[0] > generator_loss_max_in_epoch:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=True,
                        )
                    generator_loss_max_in_epoch = generator_loss[0]
                # loss 가 가장 낮은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss_min_in_epoch > generator_loss[0]:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=False,
                        )
                    generator_loss_min_in_epoch = generator_loss[0]

                # 한 배치 끝
                batch_i += self.batch_size
            # training batch 사이클 끝
            self.history[0].append(discriminator_loss[0])
            self.history[1].append(generator_loss[0])
            self.history[2].append(100 * discriminator_loss[1])
            print(f"discriminator_acces : {str(np.mean(discriminator_acces))}")
            print(
                f"Mean generator_loss : {str(np.mean(train_generator_loss))}")
            print(f"Max generator_loss : {str(np.max(train_generator_loss))}")
            print(f"Min generator_loss : {str(np.min(train_generator_loss))}")
            print(
                f"generator loss decrease : {str(self.generator_loss_previous - np.mean(train_generator_loss))}"
            )
            print(
                f"Max generator loss decrease : {str(self.generator_loss_max_previous - np.max(train_generator_loss))}"
            )
            print(
                f"current lowest generator loss : {str(self.generator_loss_min)}")

            # set look_on_for_generator_min_loss property if min loss decrease:
            if (
                self.generator_loss_min_min > generator_loss_min_in_epoch
                and self.generator_loss_max_min * 2 > generator_loss_max_in_epoch
            ):
                self.look_on_for_generator_min_loss = True
            else:
                self.look_on_for_generator_min_loss = False
            # rollback if loss not converge
            if np.mean(train_generator_loss) / self.generator_loss_min < 1.02:
                if self.generator_loss_min > np.mean(train_generator_loss):
                    # 학습중일때 진전이 너무 더디다면 learning_rate 을 조정하는 스택을 추가.
                    # loss의 감소량이 너무 적을때
                    if np.mean(train_generator_loss) / self.generator_loss_min > 0.97:
                        self.learning_rate_decay_dynamic -= (
                            self.learning_rate_decay_rate_dynamic / 10
                        )
                        print(
                            "increase Learning rate(learning_rate_increaseStack ="
                            + str(round(self.learning_rate_decay_dynamic, 3))
                            + ")"
                            + "ratio = ("
                            + str(np.mean(train_generator_loss) /
                                  self.generator_loss_min)
                            + ")"
                        )
                    self.generator_loss_min = np.mean(train_generator_loss)
                    self.generator_loss_max_min = generator_loss_max_in_epoch
                    self.generator_loss_min_min = generator_loss_min_in_epoch
                    self.weight_save_stack = True
                    self.save_study_info()
                    print("save weights")
                else:
                    # loss가 약간 증가 했을때 (정체 가능성)
                    self.learning_rate_decay_dynamic -= (
                        self.learning_rate_decay_rate_dynamic / 2
                    )
                    print(
                        "decrease Learning rate(learning_rate_increaseStack ="
                        + str(round(self.learning_rate_decay_dynamic, 3))
                        + ")"
                        + "ratio = ("
                        + str(np.mean(train_generator_loss) /
                              self.generator_loss_min)
                        + ")"
                    )
            else:
                if self.look_on_for_generator_min_loss:
                    print("min_loss is decreased. watch")
                else:
                    print("loss decreasing")
                    self.learning_rate_decay_dynamic -= (
                        self.learning_rate_decay_rate_dynamic
                    )
                    print(
                        "decrease Learning rate(learning_rate_increaseStack ="
                        + str(round(self.learning_rate_decay_dynamic, 3))
                        + ")"
                    )
                    self.load_best_weights()
            # set look_on_for_generator_min_loss property False anyway because of confusing training
            self.look_on_for_generator_min_loss = False
            # previous generator_loss 갱신
            self.generator_loss_previous = np.mean(train_generator_loss)
            self.generator_loss_max_previous = generator_loss_max_in_epoch

            if epoch >= 10 and self.weight_save_stack:
                copy(
                    "generator.h5",
                    "./generator_weights/generator_"
                    + str(round(self.generator_loss_min, 5))
                    + "_"
                    + str(round(self.generator_loss_max_min, 5))
                    + ".h5",
                )
                self.weight_save_stack = False

            self.discriminator_acc_previous = np.mean(discriminator_acces)
            self.discriminator_acc_high_indexes_previous = self.discriminator_acc_high_indexes.copy()
            train_f1_loss_list = []
            train_f1_score_list = []
            train_predict_mini_batch_size = 1
            for index in range(0, self.train_loaded_data_len, train_predict_mini_batch_size):
                
                train_original_img = self.train_loaded_data[0][index:index+train_predict_mini_batch_size]
                train_masked_img = self.train_loaded_data[1][index:index+train_predict_mini_batch_size]
                train_model_masked_img = self.generator.predict_on_batch(
                self.train_loaded_data[0][index:index+train_predict_mini_batch_size])
                
                train_f1_loss = f1_loss_for_training(train_masked_img, np.squeeze(train_model_masked_img))
                train_f1_score = f1_score(train_masked_img, np.squeeze(train_model_masked_img))
                train_f1_loss_list.append(train_f1_loss)
                train_f1_score_list.append(train_f1_score)
            print(f"train_f1_loss : {np.mean(train_f1_loss_list) * self.f1_loss_ratio}")
            print(f"train_f1_score : {1 - np.mean(train_f1_loss_list)}")
            print(f"train_f1_rounded_score : {np.mean(train_f1_score_list)}")
            
            valid_f1_loss_list = []
            valid_f1_score_list = []
            valid_predict_mini_batch_size = 1
            for index in range(0, self.valid_loaded_data_len, valid_predict_mini_batch_size):
                
                valid_original_img = self.valid_loaded_data[0][index:index+valid_predict_mini_batch_size]
                valid_masked_img = self.valid_loaded_data[1][index:index+valid_predict_mini_batch_size]
                valid_model_masked_img = self.generator.predict_on_batch(
                self.valid_loaded_data[0][index:index+valid_predict_mini_batch_size])
                
                valid_f1_loss = f1_loss_for_training(valid_masked_img, np.squeeze(valid_model_masked_img))
                valid_f1_score = f1_score(valid_masked_img, np.squeeze(valid_model_masked_img))
                valid_f1_loss_list.append(valid_f1_loss)
                valid_f1_score_list.append(valid_f1_score)
            print(f"valid_f1_loss : {np.mean(valid_f1_loss_list) * self.f1_loss_ratio}")
            print(f"train_f1_score : {1 - np.mean(valid_f1_loss_list)}")
            print(f"valid_f1_score : {np.mean(valid_f1_score_list)}")
            
    def learning_rate_scheduler(self, learning_rate, epoch):

        for step in range(0, len(self.learning_schedule)):
            if epoch < self.learning_schedule[step]:
                break
        new_learning_rate = learning_rate * (
            self.learning_rate_decay_rate_epoch ** (step)
        )
        return new_learning_rate

    def get_info_folderPath(self):
        return (
            str(round(self.generator_loss_min, 5))
            + "_"
            + str(round(self.generator_loss_max_min, 5))
            + "_"
            + str(round(self.learning_rate_decay_dynamic, 3))
        )

    def save_study_info(self, path=None):

        if path == None:
            path = self.temp_weights_path

        generator_weigth_path = os.path.join(path, "generator.h5")
        discriminator_weigth_path = os.path.join(path, "discriminator.h5")
        combined_weigth_path = os.path.join(path, "combined.h5")

        self.generator.save_weights(generator_weigth_path)
        self.discriminator.save_weights(discriminator_weigth_path)
        self.combined.save_weights(combined_weigth_path)

        study_info = {}
        study_info["start_epoch"] = self.start_epoch
        study_info["generator_loss_min"] = self.generator_loss_min
        study_info["generator_loss_max_min"] = self.generator_loss_max_min
        study_info["generator_loss_min_min"] = self.generator_loss_min_min
        study_info["learning_rate_decay_dynamic"] = self.learning_rate_decay_dynamic

        file = open(path + "/study_info.pkl", "wb")
        dump(study_info, file)
        file.close()

    def load_best_weights(self):
        self.generator.load_weights(self.temp_weights_path + "/generator.h5")
        self.discriminator.load_weights(
            self.temp_weights_path + "/discriminator.h5")
        self.combined.load_weights(self.temp_weights_path + "/combined.h5")

    def load_study_info(self):

        self.generator.load_weights("generator.h5")
        self.discriminator.load_weights("discriminator.h5")
        self.combined.load_weights("combined.h5")

        if os.path.isfile("study_info.pkl"):
            file = open("study_info.pkl", "rb")
            study_info = load(file)
            file.close()
            self.start_epoch = study_info["start_epoch"]
            self.generator_loss_min = study_info["generator_loss_min"]
            self.generator_loss_max_min = study_info["generator_loss_max_min"]
            self.generator_loss_min_min = study_info["generator_loss_min_min"]
            self.learning_rate_decay_dynamic = study_info["learning_rate_decay_dynamic"]
        else:
            print("No info pkl file!")


In [2]:
gan = Pix2PixSegmentation(generator_power=4, discriminator_power=4, generator_learning_rate=1e-4,discriminator_learning_rate=1e-4,
                          learning_rate_decay_rate_epoch = 0.5, learning_rate_decay_rate_dynamic = 0.1, test=False)

In [5]:
#gan.load_study_info()

In [3]:
#gan.find_error = True
#gan.find_error_epoch = 5

#gan.start_epoch = 50
gan.train(epochs=575, batch_size=1, sample_interval=3100)

[Epoch 0/575] [Batch 0/6200] [D loss: 0.186585, acc:  56%] [G loss: 29.575283] time: 0:00:40.926999
[Epoch 0/575] [Batch 3400/6200] [D loss: 0.000707, acc:  99%] [G loss: 33.886951] time: 0:09:36.096501
discriminator_acces : 0.9987263931766633
Mean generator_loss : 29.16664538168138
Max generator_loss : 45.14120864868164
Min generator_loss : 11.014302253723145
generator loss decrease : 70.83335461831862
Max generator loss decrease : 954.8587913513184
current lowest generator loss : 100
decrease Learning rate(learning_rate_increaseStack =-0.01)ratio = (0.29166645381681383)
save weights
train_f1_loss : 18.305668234825134
train_f1_score : 0.5996720790863037
valid_f1_loss : 18.99745464324951
valid_f1_score : 0.5981040000915527
[Epoch 1/575] [Batch 0/6200] [D loss: 0.000350, acc: 100%] [G loss: 21.291672] time: 0:20:34.587691
[Epoch 1/575] [Batch 3400/6200] [D loss: 0.315599, acc:   3%] [G loss: 29.197990] time: 0:28:25.361582
discriminator_acces : 0.35257844986454134
Mean generator_loss : 

[Epoch 12/575] [Batch 3400/6200] [D loss: 0.000595, acc: 100%] [G loss: 7.747806] time: 3:36:28.890245
discriminator_acces : 0.8408766814201108
Mean generator_loss : 11.45638964126187
Max generator_loss : 45.04446792602539
Min generator_loss : 2.52628755569458
generator loss decrease : 0.2625579951270929
Max generator loss decrease : 0.819366455078125
current lowest generator loss : 11.718947636388963
increase Learning rate(learning_rate_increaseStack =-0.03)ratio = (0.9775954289349486)
save weights
train_f1_loss : 8.748335391283035
train_f1_score : 0.6899207830429077
valid_f1_loss : 9.668775647878647
valid_f1_score : 0.6472923755645752
[Epoch 13/575] [Batch 0/6200] [D loss: 0.002609, acc: 100%] [G loss: 7.925552] time: 3:45:19.367867
[Epoch 13/575] [Batch 3400/6200] [D loss: 0.440575, acc:   0%] [G loss: 6.863770] time: 3:52:05.730511
discriminator_acces : 0.37854037377142136
Mean generator_loss : 11.135394632085678
Max generator_loss : 44.6881103515625
Min generator_loss : 2.43826031

[Epoch 23/575] [Batch 0/6200] [D loss: 0.005803, acc:  99%] [G loss: 5.744768] time: 6:21:24.123566
[Epoch 23/575] [Batch 3400/6200] [D loss: 0.012424, acc:  99%] [G loss: 8.396769] time: 6:28:09.517006
discriminator_acces : 0.7598458862304688
Mean generator_loss : 9.835857221695685
Max generator_loss : 40.87789535522461
Min generator_loss : 1.8141214847564697
generator loss decrease : 0.09655094944661613
Max generator loss decrease : -0.8380012512207031
current lowest generator loss : 9.932408171142301
increase Learning rate(learning_rate_increaseStack =-0.18)ratio = (0.9902792003929988)
save weights
train_f1_loss : 7.248540222644806
train_f1_score : 0.7199128866195679
valid_f1_loss : 8.863507211208344
valid_f1_score : 0.654345691204071
[Epoch 24/575] [Batch 0/6200] [D loss: 0.160151, acc:  14%] [G loss: 5.936010] time: 6:36:40.784602
[Epoch 24/575] [Batch 3400/6200] [D loss: 0.025636, acc:  98%] [G loss: 7.878204] time: 6:43:29.225157
discriminator_acces : 0.7917256509104083
Mean gen

[Epoch 34/575] [Batch 3400/6200] [D loss: 0.015600, acc: 100%] [G loss: 12.352178] time: 9:14:44.405542
discriminator_acces : 0.9156646236296623
Mean generator_loss : 9.125979684975839
Max generator_loss : 40.13727569580078
Min generator_loss : 1.3599821329116821
generator loss decrease : 0.04488253618440474
Max generator loss decrease : 0.8780364990234375
current lowest generator loss : 9.170862221160244
increase Learning rate(learning_rate_increaseStack =-0.33)ratio = (0.995105963310533)
save weights
train_f1_loss : 6.769692897796631
train_f1_score : 0.7356278896331787
valid_f1_loss : 8.708692342042923
valid_f1_score : 0.6585901975631714
[Epoch 35/575] [Batch 0/6200] [D loss: 0.009864, acc: 100%] [G loss: 11.535096] time: 9:23:08.568086
[Epoch 35/575] [Batch 3400/6200] [D loss: 0.013583, acc: 100%] [G loss: 11.118000] time: 9:29:52.735337
discriminator_acces : 0.9432527406754032
Mean generator_loss : 9.033680074330299
Max generator_loss : 40.377403259277344
Min generator_loss : 1.272

[Epoch 45/575] [Batch 3400/6200] [D loss: 0.005378, acc: 100%] [G loss: 5.175359] time: 12:01:01.252879
discriminator_acces : 0.9519139049899193
Mean generator_loss : 8.54524623711263
Max generator_loss : 40.345218658447266
Min generator_loss : 1.0496572256088257
generator loss decrease : 0.05111633877600319
Max generator loss decrease : 0.16552352905273438
current lowest generator loss : 8.596362575888634
increase Learning rate(learning_rate_increaseStack =-0.48)ratio = (0.994053724662641)
save weights
train_f1_loss : 6.275999546051025
train_f1_score : 0.7546578645706177
valid_f1_loss : 9.13863256573677
valid_f1_score : 0.6400792598724365
[Epoch 46/575] [Batch 0/6200] [D loss: 0.002626, acc: 100%] [G loss: 16.734127] time: 12:09:25.168808
[Epoch 46/575] [Batch 3400/6200] [D loss: 0.004647, acc: 100%] [G loss: 4.770412] time: 12:16:08.274208
discriminator_acces : 0.9151217749810988
Mean generator_loss : 8.560446612815703
Max generator_loss : 39.96366500854492
Min generator_loss : 1.086

discriminator_acces : 0.9694284156060988
Mean generator_loss : 7.684615059841064
Max generator_loss : 39.054039001464844
Min generator_loss : 1.0882444381713867
generator loss decrease : 0.06646999120712316
Max generator loss decrease : -0.4720306396484375
current lowest generator loss : 7.751085051048187
increase Learning rate(learning_rate_increaseStack =-0.66)ratio = (0.9914244275776416)
save weights
train_f1_loss : 5.594632029533386
train_f1_score : 0.7814393043518066
valid_f1_loss : 8.88945460319519
valid_f1_score : 0.649445116519928
[Epoch 57/575] [Batch 0/6200] [D loss: 0.043787, acc: 100%] [G loss: 7.598090] time: 14:58:04.089699
[Epoch 57/575] [Batch 3400/6200] [D loss: 0.011821, acc: 100%] [G loss: 8.102150] time: 15:04:56.378201
discriminator_acces : 0.982198023642263
Mean generator_loss : 7.649572899149311
Max generator_loss : 38.63017272949219
Min generator_loss : 1.3202741146087646
generator loss decrease : 0.03504216069175303
Max generator loss decrease : 0.4238662719726

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gr300\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-8ff6495d493e>", line 5, in <module>
    gan.train(epochs=575, batch_size=1, sample_interval=3400)
  File "<ipython-input-1-3f95f3fef21a>", line 314, in train
    generator_loss = self.combined.train_on_batch(
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1695, in train_on_batch
    logs = train_function(iterator)
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\eager\def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\eager\def_function.py", line 807, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "C:

TypeError: object of type 'NoneType' has no len()

In [19]:
np.mean(gan.discriminator_acc_high_indexes_previous[0:1])

0.54559326171875

In [ ]:
a = list(range(3))

print(a[1,3])

In [ ]:
gan.discriminator.test_on_batch([original_img, predicted_img], gan.valid_path)

In [ ]:
gan.combined.train_on_batch(
    [original_img, predicted_img],
    [gan.valid_path, masked_img],
)

In [ ]:
gan.loaded_data_index

In [ ]:
print(gan.loaded_data[1][0])

In [ ]:
gan.model_masked_img[:,:,:,0][0]

In [ ]:
cv2.cvtColor(gan.model_masked_img[:,:,:,0][0], cv2.COLOR_GRAY2RGB).shape

In [ ]:
gan.load_study_info()

In [ ]:
print(gan.original_img.shape)
print(gan.model_masked_img.shape)
print(gan.masked_img.shape)

In [ ]:
temp_origin = ((gan.original_img[0]+1) * 127.5).astype('uint8')
temp_masked = ((gan.masked_img[0]+1) * 127.5).astype('uint8')

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(temp_origin)
plt.show()

In [ ]:
plt.imshow(gan.loaded_data[1][0])
plt.show()

In [ ]:
from glob import glob

images = glob("C:\\Users\\gr300\\Desktop\\Works\의료데이터\\Level_0_512_random_Split\\FOLD_1\\wo_SN\\slide-2020-04-22T09-53-31-R3-S20\\image_MONO_random\\*")

print(len(images))

In [ ]:
def my_gen